# *Customer-Support-Chatbot*

### Intsalling selenium

pip install unstructured selenium

## Defining Dependencies

In [33]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate
import os
from langchain.embeddings import  HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
    


## Defenining Open AI API Key

In [44]:
os.environ["OPENAI_API_KEY"] = "KEY"

## Defining URLS Paths to Get the Required Articals

In [8]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

## Initializing SeleniumURLLoader and Extracting The Content

In [45]:
loader = SeleniumURLLoader(urls=urls)
docs_not_splitted = loader.load()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Defining The Charater Splitting Parameters and Splitting the Scrapped Content

In [9]:

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Created a chunk of size 1226, which is longer than the specified 1000


## verifing  whether the content has been loaded sucessfully or not


In [ ]:
docs[0]

## Importing  Embeddings Module

In [16]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Creating Vector Database Using FAISS for Faster Similarity Search

In [19]:
faiss_db = FAISS.from_documents(docs, embeddings)

In [36]:
result = faiss_db.similarity_search("how to delete spotify accounts")


## Giving memory to the Model to concerntrate to the context of the user

In [37]:
# Create a ConversationBufferMemory instance.
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

# Load the QA chain with memory.
chain = load_qa_chain(OpenAI(), chain_type="stuff", memory=memory)

## Defining the Function to perform Question and Answering

In [41]:
def answer_question(query):
    """
    Answer a question using the loaded model and preprocessed embeddings.
    """
    # Search for documents similar to the query within the preprocessed embeddings
    docs = faiss_db.similarity_search(query)

    # Get the answer from the model
    answer = chain.run(input_documents=docs, question=query)
    return answer

In [42]:
result = answer_question("What Happens When You Delete Your Spotify Account")

In [43]:
print(result)

 When you permanently delete your Spotify account, all of your data will be erased forever and you won’t have access to any of your Spotify data anymore. You will lose access to your playlists and saved music, liked songs, followers, and followings. You will also not be able to use your Spotify username again in the future.
